In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
Cloning into 'mmdetection'...
remote: Enumerating objects: 24303, done.
remote: Total 24303 (delta 0), reused 0 (delta 0), pack-reused 24303
Receiving objects: 100% (24303/24303), 37.44 MiB | 3.23 MiB/s, done.
Resolving deltas: 100% (16965/16965), done.
/content/mmdetection/mmdetection
Obtaining file:///content/mmdetection/mmdetection
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.23.0
    Can't uninstall 'mmdet'. No files were found to uninstall.
  Running setup.py develop for mmdet


In [ ]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.23.0
11.1
GCC 7.3


In [ ]:
%cd ..
%cd ..

/
/


In [ ]:
!mkdir content/mmdetection/data

mkdir: cannot create directory ‘content/mmdetection/data’: File exists


In [ ]:
!cp -a content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/* content/mmdetection/data

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/coco_detection_base.py content/mmdetection/configs/_base_/datasets

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_fpn_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [ ]:
%cd content/mmdetection

/content/mmdetection


In [ ]:
%pwd

'/content/mmdetection'

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py')
print(cfg)

Config (path: ./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': False}, 'norm_eval': True, 'style': 'caffe', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron2/resnet50_caffe'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoI

In [ ]:
print(cfg.dataset_type)
print(cfg.data_root)
print(cfg.data.test.type)
#print(cfg.data.test.data_root)
print(cfg.data.test.ann_file)
print(cfg.data.test.img_prefix)
# cfg.data.test.ann_file = 'data/coco/instances_test2017.json'
# cfg.data.test.img_prefix = 'data/coco/test/'

CocoDataset
data/coco/
CocoDataset
data/coco/instances_test2017.json
data/coco/test/


In [ ]:
print(cfg.data.train.type)
#print(cfg.data.train.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
# cfg.data.train.ann_file = 'data/coco/instances_train2017_trunc.json'
# cfg.data.train.img_prefix = 'data/coco/train'
# print(cfg.data.train.ann_file)
# print(cfg.data.train.img_prefix)

CocoDataset
data/coco/instances_train2017_trunc.json
data/coco/train/


In [ ]:
print(cfg.data.val.type)
#print(cfg.data.val.data_root)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix)
# cfg.data.val.ann_file = 'data/coco/instances_val2017_trunc.json'
# cfg.data.val.img_prefix = 'data/coco/val'
# print(cfg.data.val.ann_file)
# print(cfg.data.val.img_prefix)

CocoDataset
data/coco/instances_val2017_trunc.json
data/coco/val/


In [ ]:
# modify num classes of the model in box head
print(cfg.model.roi_head.bbox_head.num_classes)
# If we need to finetune a model based on a pre-trained detector, we need to
# use load_from to set the path of checkpoints.
print(cfg.load_from)

80
None


In [ ]:
# Set up working dir to save files and logs.
#print(cfg.work_dir)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02/8
#cfg.lr_config.warmup = None
cfg.log_config.interval = 10

print(cfg.optimizer.lr)
print(cfg.lr_config.warmup)
print(cfg.log_config.interval)
# Change the evaluation metric since we use customized dataset.
print(cfg.evaluation.metric)
cfg.evaluation.metric = 'bbox'
print(cfg.evaluation.metric)
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
print(cfg.evaluation.interval)
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12
print(cfg.checkpoint_config.interval)

# Set seed thus the results are more reproducible
#print(cfg.seed)
#set_random_seed(0, deterministic=False)
#print(cfg.gpu_ids)

print(cfg.log_config.hooks)

0.0025
linear
10
bbox
bbox
12
12
[{'type': 'TextLoggerHook'}]


In [ ]:
from mmdet.apis import set_random_seed

# # use load_from to set the path of checkpoints.
# cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


In [113]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=20.45s)
creating index...
index created!
loading annotations into memory...


2022-04-23 19:17:14,208 - mmdet - INFO - Start running, host: root@5b5438d06431, work_dir: /content/mmdetection/tutorial_exps
2022-04-23 19:17:14,210 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook   

Done (t=5.82s)
creating index...
index created!


2022-04-23 19:17:19,882 - mmdet - INFO - Epoch [1][10/1981]	lr: 4.746e-05, eta: 3:39:01, time: 0.553, data_time: 0.252, memory: 5954, loss_rpn_cls: 1.1270, loss_rpn_bbox: 1.6368, loss_cls: 0.5587, acc: 90.9863, loss_bbox: 0.1225, loss: 3.4449
2022-04-23 19:17:22,842 - mmdet - INFO - Epoch [1][20/1981]	lr: 9.741e-05, eta: 2:47:54, time: 0.295, data_time: 0.019, memory: 5954, loss_rpn_cls: 1.0670, loss_rpn_bbox: 1.1880, loss_cls: 0.7641, acc: 96.9727, loss_bbox: 0.2119, loss: 3.2309
2022-04-23 19:17:25,825 - mmdet - INFO - Epoch [1][30/1981]	lr: 1.474e-04, eta: 2:31:09, time: 0.298, data_time: 0.020, memory: 5954, loss_rpn_cls: 0.9510, loss_rpn_bbox: 1.1339, loss_cls: 0.6740, acc: 97.9199, loss_bbox: 0.1645, loss: 2.9233
2022-04-23 19:17:28,973 - mmdet - INFO - Epoch [1][40/1981]	lr: 1.973e-04, eta: 2:24:39, time: 0.317, data_time: 0.020, memory: 5954, loss_rpn_cls: 0.7432, loss_rpn_bbox: 1.1711, loss_cls: 0.3932, acc: 97.4316, loss_bbox: 0.1346, loss: 2.4422
2022-04-23 19:17:32,178 - mm

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 12.9 task/s, elapsed: 77s, ETA:     0s

2022-04-23 21:18:22,012 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.04s).
Accumulating evaluation results...


2022-04-23 21:18:30,309 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.014
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.01s).


In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=31.73s)
creating index...
index created!
loading annotations into memory...


2022-04-23 13:24:50,981 - mmdet - INFO - Start running, host: root@5b5438d06431, work_dir: /content/mmdetection/tutorial_exps
2022-04-23 13:24:50,982 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook   

Done (t=0.60s)
creating index...
index created!


2022-04-23 13:24:56,606 - mmdet - INFO - Epoch [1][10/1981]	lr: 4.746e-05, eta: 3:37:29, time: 0.549, data_time: 0.250, memory: 5864, loss_rpn_cls: 1.1270, loss_rpn_bbox: 1.6368, loss_cls: 0.5577, acc: 91.0742, loss_bbox: 0.1225, loss: 3.4439
2022-04-23 13:24:59,599 - mmdet - INFO - Epoch [1][20/1981]	lr: 9.741e-05, eta: 2:47:44, time: 0.298, data_time: 0.019, memory: 5864, loss_rpn_cls: 1.0654, loss_rpn_bbox: 1.1875, loss_cls: 0.7673, acc: 96.9531, loss_bbox: 0.2132, loss: 3.2333
2022-04-23 13:25:02,619 - mmdet - INFO - Epoch [1][30/1981]	lr: 1.474e-04, eta: 2:31:33, time: 0.302, data_time: 0.019, memory: 5864, loss_rpn_cls: 0.9371, loss_rpn_bbox: 1.1311, loss_cls: 0.6768, acc: 97.8711, loss_bbox: 0.1653, loss: 2.9104
2022-04-23 13:25:05,779 - mmdet - INFO - Epoch [1][40/1981]	lr: 1.973e-04, eta: 2:25:07, time: 0.319, data_time: 0.020, memory: 5864, loss_rpn_cls: 0.7395, loss_rpn_bbox: 1.1517, loss_cls: 0.3817, acc: 97.5098, loss_bbox: 0.1348, loss: 2.4076
2022-04-23 13:25:08,973 - mm

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 13.0 task/s, elapsed: 77s, ETA:     0s

KeyError: ignored

In [ ]:
!python tools/test.py configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py tutorial_exps/epoch_12.pth --out results.pkl --eval bbox

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=22.81s)
creating index...
index created!
loading annotations into memory...


2022-04-23 02:47:21,394 - mmdet - INFO - Start running, host: root@d4e9063d71dc, work_dir: /content/mmdetection/tutorial_exps
2022-04-23 02:47:21,396 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook   

Done (t=0.59s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2022-04-23 02:47:45,167 - mmdet - INFO - Epoch [1][50/1981]	lr: 1.978e-03, eta: 2:49:23, time: 0.428, data_time: 0.054, memory: 3620, loss_rpn_cls: 0.7528, loss_rpn_bbox: 1.3381, loss_cls: 1.1024, acc: 91.8066, loss_bbox: 0.2193, loss: 3.4127
2022-04-23 02:48:00,609 - mmdet - INFO - Epoch [1][100/1981]	lr: 3.976e-03, eta: 2:25:28, time: 0.309, data_time: 0.009, memory: 3620, loss_rpn_cls: 0.3511, loss_rpn_bbox: 0.2203, loss_cls: 0.5503, acc: 95.4473, loss_bbox: 0.1938, loss: 1.3156
2022-04-23 02:48:15,913 - mmdet - INFO - Epoch [1][150/1981]	lr: 5.974e-03, eta: 2:17:00, time: 0.307, data_t

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 17.0 task/s, elapsed: 59s, ETA:     0s

2022-04-23 04:30:06,535 - mmdet - INFO - Evaluating bbox...
2022-04-23 04:30:06,537 - mmdet - ERROR - The testing results of the whole dataset is empty.
2022-04-23 04:30:06,548 - mmdet - INFO - Epoch(val) [12][1000]	


Loading and preparing results...


In [ ]:
!python tools/test.py configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py tutorial_exps/epoch_12.pth --out results.pkl --eval bbox

/content/mmdetection/mmdet/utils/setup_env.py:33: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/content/mmdetection/mmdet/utils/setup_env.py:43: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
loading annotations into memory...
tcmalloc: large alloc 1714692096 bytes == 0x55dff8de4000 @  0x7f9afbfbd1e7 0x55dff0bf2338 0x55dff0bbcad7 0x55dff0d3b575 0x55dff0cd58b8 0x55dff0bc03a2 0x55dff0c9ec6e 0x55dff0cd5832 0x55dff0bc03a2 0x55dff0bc35f6 0x55dff0d3ec03 0x55dff0bbfff9 0x55dff0cb1d4d 0x55dff0c33ec8 0x55dff0c2ecdd 